In [2]:
from FundData import *
from helper import Helper
import pprint, json
import fitz

#dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
#fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'


mutual_fund = Helper.get_fund_paths(fund_path)

In [54]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        print(f"___{pgn}___")
        for num,block in enumerate(blocks.get('blocks',[])):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if num in range(3,10):
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
        doc = fitz.open(path)
        final_fund_names = dict()
        
        for pgn in range(doc.page_count):
            fund_names = ''
            if pgn in pages:
                page = doc[pgn]            
                blocks = page.get_text("dict")['blocks']
                
                sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
                for count,block in enumerate(sorted_blocks):
                    for line in block.get("lines", []):
                        for span in line.get("spans", []):
                            text = span['text'].strip()
                            if count in range(0,2): #contains fund name        
                                fund_names += f'{text} '
            if matches:= re.search(r'\bMahindra.*?(Fund|ETF|EOF|FOF|FTF|Path)\b', fund_names, re.IGNORECASE):           
                final_fund_names[pgn] = matches.group()
            else:
                final_fund_names[pgn] = ''

        return final_fund_names

In [60]:
# Lines to be drawn (start and end points)
lines = [
    ((220, 0), (220, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0, 145, 180, 812)]

sample_path  = mutual_fund["Baroda BNP Paribas Mutual Fund"]
pages = [i for i in range(1,200,1)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)
data = Helper.get_all_pdf_data(sample_path)

Modified PDF saved to: C:\Users\rando\OneDrive\Documents\mywork-repo\output\DryRun.pdf


In [61]:
data[6]

{'pgn': 6,
 'blocks': [{'number': 4,
   'type': 0,
   'bbox': (373.8619689941406,
    20.36587905883789,
    557.7525634765625,
    27.11939811706543),
   'lines': [{'spans': [{'size': 6.743217945098877,
       'flags': 0,
       'font': 'Unnamed-T3',
       'color': -16777216,
       'ascender': 0.7500000596046448,
       'descender': -0.242000013589859,
       'text': 'Scheme Riskometer^^',
       'origin': (373.8619689941406, 25.47437858581543),
       'bbox': (373.8619689941406,
        20.36587905883789,
        437.7135009765625,
        27.11939811706543)}],
     'wmode': 0,
     'dir': (1.0, 0.0),
     'bbox': (373.8619689941406,
      20.36587905883789,
      437.7135009765625,
      27.11939811706543)},
    {'spans': [{'size': 6.743217945098877,
       'flags': 0,
       'font': 'Unnamed-T3',
       'color': -16777216,
       'ascender': 0.7500000596046448,
       'descender': -0.242000013589859,
       'text': 'Benchmark (Tier 1) Riskometer^',
       'origin': (464.520812988